In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
from pathlib import Path
from typing import Tuple

import numpy as np
from tqdm import tqdm_notebook as tqdm
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import patches
rcParams['figure.figsize'] = 10, 10

In [ ]:
base_path = Path('/media/klaus/Ondrag/dev/datasets/300VW_Dataset_2015_12_14/')
padding = 50
expand_ratio = 0.0
n_points = 68

In [ ]:
def plot(image, points: np.ndarray = None, box: Tuple[int, int, int, int] = None) -> None:
    plt.figure()
    plt.imshow(image)
    plt.axis('off')

    if points is not None:
        plt.plot(points[:, 0], points[:, 1], 'gx')
    
    if box is not None:
        x1, y1, x2, y2 = box
        rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, 
                                 linewidth=1, edgecolor='r', facecolor='none')
        ax = plt.gca()
        ax.add_patch(rect)
    
    plt.show()    

    

def load_pts_file(file_path: Path) -> np.ndarray:
    with open(str(file_path), 'r') as file:
        lines = file.readlines()
        
    assert lines[0] == 'version: 1\n'
    assert lines[1] == f'n_points: {n_points}\n'
    
    lines = [l.strip() for l in lines]
    # remove
    # version: 1
    # n_points: 68
    # {
    lines = lines[3:]
    # remove
    # }
    lines = lines[:-1]
    points = [[float(x) for x in p.split()]
              for p in lines]
    points = np.asarray(points)
    assert points.shape == (n_points, 2)
    return points


def points_to_box(points: np.ndarray) -> Tuple[int, int, int, int]:
    x1, y1, x2, y2 = [points[:, 0].min(), points[:, 1].min(), points[:, 0].max(), points[:, 1].max()]
    x1, y1 = [t - padding for t in (x1, y1)]
    x2, y2 = [t + padding for t in (x2, y2)]
    W, H = x2 - x1, y2 - y1
    assert W > 0 and H > 0
    
    if expand_ratio is not None:
        x1 = math.floor(x1 - W * expand_ratio)
        y1 = math.floor(y1 - H * expand_ratio)
        x2 = math.ceil(x2 + W * expand_ratio)
        y2 = math.ceil(y2 + H * expand_ratio)
    
    x1, y1, x2, y2 = [int(t) for t in (x1, y1, x2, y2)]
    return x1, y1, x2, y2

In [ ]:
all_videos = sorted([p for p in base_path.iterdir() if p.is_dir()])
assert len(all_videos) == 114
for video_path in tqdm(all_videos, desc='video'):
    all_frames_paths = sorted(list((video_path / 'extraction').glob('*.png')))
    all_annotations_paths = sorted(list((video_path / 'annot').glob('*.pts')))
    assert len(all_frames_paths) == len(all_annotations_paths)
    is_first_frame = True
    for frame_path, annotation_path in tqdm(list(zip(all_frames_paths, all_annotations_paths)), desc='frame', 
                                            leave=False):
        assert frame_path.stem == annotation_path.stem
        
        image = mpimg.imread(str(frame_path))
        points = load_pts_file(str(annotation_path))
        box = points_to_box(points)
        # plot(image)
        # plot(image, points)
        if is_first_frame:
            is_first_frame = False
            plot(image, points, box)
            break